In [1]:
%pip install langchain langchain-openai langchain-community langchain-core openai pypdf azure-keyvault-secrets azure-identity faiss-cpu dotenv

  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> [32 lines of output]
        Using cached distribute-0.7.3.zip (145 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'error'
        error: subprocess-exited-with-error
      
        Ã— Preparing metadata (pyproject.toml) did not run successfully.
        â”‚ exit code: 1
        â•°â”€> [6 lines of output]
            usage: setup.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
               or: setup.py --help [cmd1 cmd2 ...]
               or: setup.py --help-commands
               or: setup.py cmd --help
      
            

In [4]:
import json
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import LLMChain
from langchain_openai import AzureChatOpenAI

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
deployment_name = "uptale-gpt-35-turbo"

llm = AzureChatOpenAI(
    azure_deployment=deployment_name
)

In [7]:
def process_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    return texts

In [8]:
def generate_title(texts):
    combined_text = " ".join(text.page_content for text in texts)
    prompt = PromptTemplate(
        input_variables=["document"],
        template="Create a catchy title for a 360-degree educational experience based on it."
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(document=combined_text)
    return response.strip()

In [9]:
def generate_scenes(texts, num_scenes=5):
    combined_text = " ".join(text.page_content for text in texts)
    scenes = []
    for i in range(num_scenes):
        prompt = PromptTemplate(
            input_variables=["scene_number", "document"],
            template="""
            Create scene {scene_number} for a 360-degree educational experience based on the document content.
            Include:
            1. A brief description of the scene
            2. Educational content relevant to the document
            3. A transition to the next scene (use one of the following):
               - Tag Door => Target Scene
               - Tag QCM => Go to Scene (with up to 4 multiple-choice answers)
               - Tag Microphone Rules => Allow to trigger an action to go to another scene

            Format the output as a JSON object.
            """
        )
        chain = LLMChain(llm=llm, prompt=prompt)
        scene = chain.run(scene_number=i+1, document=combined_text)
        try:
            scenes.append(json.loads(scene))
        except json.JSONDecodeError:
            print(f"Error parsing JSON for scene {i+1}. Skipping this scene.")
    return scenes

In [10]:
def generate_360_experience(pdf_path):
    texts = process_pdf(pdf_path)
    title = generate_title(texts)
    scenes = generate_scenes(texts)

    experience = {
        "title": title,
        "scenes": scenes
    }

    return json.dumps(experience, indent=2)

In [11]:
pdf_path = "./content/example1.pdf"
experience_json = generate_360_experience(pdf_path)
print(experience_json)

with open("./content/360_experience.json", "w") as f:
    f.write(experience_json)

C:\Users\abdes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
C:\Users\abdes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{
  "title": "\"Immersive Learning: The Journey to 360-Degree Education\"",
  "scenes": [
    {
      "scene1": {
        "description": "Scene 1: Introduction to Climate Change",
        "content": "This scene is set in a virtual classroom where the teacher is giving a lecture on climate change. The students are seated in a circular arrangement around the teacher, allowing them to look around and observe their surroundings. The teacher explains the basic concepts of climate change, such as the greenhouse effect, global warming, and the role of human activities in contributing to climate change.",
        "transition": {
          "tag": "Tag QCM",
          "answers": [
            "A. What is the main cause of climate change?",
            "B. How does climate change affect biodiversity?",
            "C. What are the potential solutions to mitigate climate change?",
            "D. How can individuals reduce their carbon footprint?"
          ],
          "targetScene": "scene2"
   

 # CSM Agent

In [11]:
from langchain.agents import Tool, AgentExecutor, ZeroShotAgent
from langchain.memory import ConversationBufferMemory

In [12]:
def generate_title_tool(chunks, csm_instructions):
    combined_text = " ".join(chunk.page_content for chunk in chunks[:2])
    prompt = PromptTemplate(
        input_variables=["document", "instructions"],
        template="Create a catchy title for a 360-degree educational experience based on the following document, while adhering to these instructions: {instructions}\n\nDocument: {document}"
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(document=combined_text, instructions=csm_instructions)
    return response.strip()

def generate_scenes_tool(chunks, csm_instructions, num_scenes=5):
    scenes = []
    for i in range(num_scenes):
        chunk = chunks[i] if i < len(chunks) else chunks[-1]
        prompt = PromptTemplate(
            input_variables=["scene_number", "document", "instructions"],
            template="""
            Create scene {scene_number} for a 360-degree educational experience based on the document content, while adhering to these instructions: {instructions}

            Include:
            1. A brief description of the scene (3 lines maximum)
            2. Educational content relevant to the document
            3. A transition to the next scene (use one of the following):
               - Tag Door => Target Scene
               - Tag QCM => Go to Scene (with up to 4 multiple-choice answers)
               - Tag Microphone Rules => Allow to trigger an action to go to another scene

            Format the output as a JSON object.
            """
        )
        chain = LLMChain(llm=llm, prompt=prompt)
        scene = chain.run(scene_number=i+1, document=chunk.page_content, instructions=csm_instructions)
        try:
            scenes.append(json.loads(scene))
        except json.JSONDecodeError:
            print(f"Error parsing JSON for scene {i+1}. Skipping this scene.")
    return scenes

def evaluate_experience_tool(experience, csm_instructions):
    prompt = PromptTemplate(
        input_variables=["experience", "instructions"],
        template="""
        Evaluate the following 360-degree educational experience based on these instructions: {instructions}

        Experience:
        {experience}

        Provide a score from 1 to 10 and a brief explanation (max 50 words) of your evaluation.
        Return your response as a JSON object with 'score' and 'explanation' keys.
        Example: {{"score": 8, "explanation": "The experience is engaging and well-structured."}}
        """
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    evaluation = chain.run(experience=json.dumps(experience), instructions=csm_instructions)
    
    try:
        json_start = evaluation.index('{')
        json_end = evaluation.rindex('}') + 1
        json_str = evaluation[json_start:json_end]
        return json.loads(json_str)
    except (ValueError, json.JSONDecodeError):
        print("Failed to parse evaluation result. Using default evaluation.")
        return {"score": 0, "explanation": "Failed to generate a valid evaluation."}

In [13]:
import json

def run_csm_agent(pdf_path, csm_instructions_path):
    chunks = process_pdf(pdf_path)
    
    with open(csm_instructions_path, 'r') as file:
        csm_instructions = file.read()

    tools = [
        Tool(
            name="GenerateTitle",
            func=lambda _: generate_title_tool(chunks, csm_instructions),
            description="Generates a title for the 360-degree experience"
        ),
        Tool(
            name="GenerateScenes",
            func=lambda _: generate_scenes_tool(chunks, csm_instructions),
            description="Generates scenes for the 360-degree experience"
        ),
        Tool(
            name="EvaluateExperience",
            func=lambda experience: evaluate_experience_tool(experience, csm_instructions),
            description="Evaluates the generated 360-degree experience"
        )
    ]

    prefix = """You are an AI assistant tasked with creating and evaluating a 360-degree educational experience based on provided content and instructions. Follow the CSM instructions carefully."""
    suffix = """Begin!

Question: {input}
{agent_scratchpad}"""

    prompt = ZeroShotAgent.create_prompt(
        tools, 
        prefix=prefix, 
        suffix=suffix, 
        input_variables=["input", "agent_scratchpad"]
    )

    llm_chain = LLMChain(llm=llm, prompt=prompt)
    memory = ConversationBufferMemory(memory_key="agent_scratchpad")
    agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools)
    agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

    experience = None
    evaluation = {"score": 0}

    while evaluation["score"] < 7:
        experience = {}
        
        experience["title"] = agent_executor.run("Generate a title for the 360-degree educational experience.")
        
        experience["scenes"] = agent_executor.run("Generate 5 scenes for the 360-degree educational experience.")
        
        evaluation_output = agent_executor.run(f"Evaluate the following 360-degree educational experience:\n{json.dumps(experience)}")
        print(f"Raw Evaluation Output: {evaluation_output}")

        try:
            evaluation = json.loads(evaluation_output)
        except json.JSONDecodeError:
            print("Failed to parse evaluation output. Regenerating the experience...")
            continue
        
        print(f"Current evaluation score: {evaluation['score']}")
        
        if evaluation["score"] < 7:
            print("Score below 7. Regenerating the experience...")
        else:
            print("Score 7 or above. Returning the final experience.")

    return experience

print(run_csm_agent("./content/test_csm.pdf", "./content/csm_instructions.txt"))

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)
Ignoring wrong pointing object 47 0 (offset 0)
Ignoring wrong pointing object 49 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 53 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 57 0 (offset 0)
Ignoring wrong 



> Entering new AgentExecutor chain...
Thought: I need to come up with a catchy and informative title for the 360-degree educational experience.
Action: GenerateTitle
Action Input: None
Observation: "A Breath of Knowledge: Exploring Respiratory Infections in a 360-Degree Educational Experience"
Thought:This title combines the idea of learning and exploring respiratory infections in a catchy way. It also highlights the immersive nature of the educational experience.
Final Answer: "A Breath of Knowledge: Exploring Respiratory Infections in a 360-Degree Educational Experience"

> Finished chain.


> Entering new AgentExecutor chain...
Thought: To generate 5 scenes for the 360-degree educational experience, I need to understand the content and instructions provided.

Action: GenerateScenes
Action Input: 5
Observation: [{'scene1': {'description': 'Welcome to the VR experience! You are standing in a lush forest surrounded by tall trees.', 'educational_content': 'This scene introduces the us

KeyboardInterrupt: 